In [38]:
import pandas as pd

df = pd.read_csv('../raw_data/enrollment_data_by_age_group.csv')

new_rows = []
for params, sdf in df.groupby(['Season', 'Age_group']):
    season, ag = params

    try:
        vac_enrolled = sdf[(sdf.Vaccination_status == 1) & (sdf.Refused_enrollment == 0)].Count.iloc[0, ]
    except IndexError:
        vac_enrolled = 0
    try:
        vac_unenrolled = sdf[(sdf.Vaccination_status == 1) & (sdf.Refused_enrollment == 1)].Count.iloc[0, ]
    except IndexError:
        vac_unenrolled = 0
    try:
        unvac_enrolled = sdf[(sdf.Vaccination_status == 0) & (sdf.Refused_enrollment == 0)].Count.iloc[0, ]
    except IndexError:
        unvac_enrolled = 0
    try:
        unvac_unenrolled = sdf[(sdf.Vaccination_status == 0) & (sdf.Refused_enrollment == 1)].Count.iloc[0, ]
    except IndexError:
        unvac_unenrolled = 0
    
    try:
        vac_enroll_fraction = vac_enrolled / (vac_enrolled + vac_unenrolled)
    except ZeroDivisionError:
        vac_enroll_fraction = 0
    try:
        unvac_enroll_fraction = unvac_enrolled / (unvac_enrolled + unvac_unenrolled)
    except ZeroDivisionError:
        unvac_enroll_fraction = 0
    new_rows.append([season, ag, unvac_enroll_fraction, vac_enroll_fraction])

raw_fractions = pd.DataFrame(new_rows, columns = ['Season', 'Age group', 'Unvaccinated', 'Vaccinated'])
averages = raw_fractions.groupby('Age group').mean()

seasons = list(range(2008, 2019)) + [2009.5]

final_rows = []
for season in seasons:
    if season == 2009.5:
        sstring = '2009pan'
    else:
        sstring = str(int(season))
        
    if season in [2008, 2009]:
        sdf = averages
    else:
        sdf = raw_fractions[raw_fractions.Season == sstring]
        sdf.index = sdf['Age group']
        sdf = sdf[['Unvaccinated', 'Vaccinated']]
    
    sdf = sdf / sdf.loc['20-29', 'Unvaccinated']
    
    for ag, row in sdf.iterrows():
        if ag == '65+':
            ag_new = '65-100'
        else:
            ag_new = ag
        final_rows.append([season, ag_new, row.Unvaccinated, row.Vaccinated])

norm_fractions = pd.DataFrame(final_rows, columns = ['Season', 'Age group', 'Unvaccinated', 'Vaccinated'])
norm_fractions.to_csv('../data/normalized_enrollment.csv', index=False)